In [1]:
from tensoraerospace.agent.pid import PID
import numpy as np
from ray import train, tune

from tensoraerospace.envs.f16.linear_longitudial import LinearLongitudinalF16
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

import gymnasium as gym 
from tensoraerospace.benchmark.function import overshoot, settling_time, static_error

In [2]:
dt = 0.1  # Дискретизация
tp = generate_time_period(tn=120, dt=dt) # Временной периуд
tps = convert_tp_to_sec_tp(tp, dt=dt)
number_time_steps = len(tp) # Количество временных шагов
reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=10, output_rad=False), [1, -1]) # Заданный сигнал

In [43]:
def env_optimization(ki, kp, kd):
    """
    Оптимизация среды моделирования для настройки коэффициентов ПИД-регулятора.

    Эта функция оптимизирует параметры ПИД-регулятора (ки, кп, кд), используя модельную среду.
    Она оценивает качество настройки по критериям статической ошибки, перерегулирования и времени установления.

    Args:
        ki (float): Коэффициент интегральной составляющей.
        kp (float): Коэффициент пропорциональной составляющей.
        kd (float): Коэффициент дифференциальной составляющей.

    Returns:
        float: Суммарная оценка качества настройки регулятора.

    """

    # Инициализация истории и настройка параметров времени
    hist = []
    dt = 0.1
    tp = generate_time_period(tn=80, dt=dt)
    tps = convert_tp_to_sec_tp(tp, dt=dt)
    number_time_steps = len(tp)

    # Создание заданного сигнала
    reference_signals = np.reshape(unit_step(degree=5, tp=tp, time_step=0.5, output_rad=False), [1, -1])

    # Настройка модельной среды
    env = gym.make('LinearLongitudinalB747-v0',
                number_time_steps=number_time_steps, 
                initial_state=[[0],[0],[0],[0]],
                reference_signal=reference_signals,
                state_space = [ "u", "w", "q", "theta"],
                output_space = ["u", "w", "q", "theta"],
                tracking_states=["theta"], use_reward = False)
    env.reset()

    # Инициализация ПИД-регулятора
    pid = PID(kp=kp, ki=ki, kd=kd, dt=dt, env=env)
    xt = np.array([0,0,0,0])

    # Цикл моделирования среды
    for step in range(number_time_steps - 2):
        setpoint = reference_signals[0, step]
        hist.append(xt[3])
        ut = pid.select_action(setpoint, xt[3])
        xt, reward, terminated, truncated, info = env.step(np.array([ut.item()]))

    # Условие для досрочного прекращения оптимизации
    if 6 < max(hist):
        return 10000

    # Получение исходных сигналов для анализа
    system_signal_orig = env.unwrapped.model.get_state('theta', to_deg=False)[:797]
    control_signal_orig = reference_signals[0][:797]

    # Расчет оценочной функции
    return np.abs(static_error(control_signal_orig, system_signal_orig)) * 0.5 + \
           np.abs(overshoot(control_signal_orig, system_signal_orig)) ** 0.8 + \
           (settling_time(control_signal_orig, system_signal_orig) * dt) ** 0.1


In [44]:
env_optimization(1, 1, 1)

84.68317101604991

In [45]:
def easy_objective(config):
    """
    Функция цели для оптимизатора.

    Принимает конфигурацию с параметрами ПИД-регулятора и возвращает оценку качества настройки.

    Args:
        config (dict): Словарь с параметрами ПИД-регулятора (ki, kp, kd).

    """

    # Извлечение гиперпараметров из конфигурации
    ki, kp, kd = config["ki"], config["kp"], config["kd"]

    # Вычисление оценки
    intermediate_score = env_optimization(ki, kp, kd)

    # Отчет о текущем значении потерь
    train.report({"mean_loss": intermediate_score})

In [52]:
# Настройка и запуск оптимизатора
tuner = tune.Tuner(
        easy_objective,
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            num_samples=7000,
        ),
        param_space={
            "ki": tune.uniform(-1, 0),
            "kp": tune.uniform(-100, 0),
            "kd": tune.uniform(-100, 0),
        },
    )

In [53]:
# Выполнение оптимизации
results = tuner.fit()

2024-08-21 04:36:13,383	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-21 04:36:38,985	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-21 04:36:52,149	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-21 04:36:52,841	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-21 04:36:53,057	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UN

In [51]:
# Получение лучших результатов оптимизации
results.get_best_result().config

{'ki': -0.0022143097088502817,
 'kp': -4.291235281088618,
 'kd': -71.06391797487251}

In [42]:
# Сортировка и получение значений средних потерь
results.get_dataframe()['mean_loss'].sort_values()

712        4.076111
907        4.282368
859        5.492190
189        7.436287
290        8.263364
           ...     
342    10000.000000
343    10000.000000
344    10000.000000
346    10000.000000
999    10000.000000
Name: mean_loss, Length: 1000, dtype: float64